In [34]:
# Importing necessary libraries 
import os
import json
import pandas as pd
import numpy as np
import glob
from IPython import get_ipython


# For data cleaning and visualization
import pandas as pd
import numpy as np
from numpy import array
from datetime import date
from datetime import datetime, timedelta
import pandas_datareader.data as web


# For saving the model
import pickle

# For model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import math

import re
import nltk
from nltk.corpus import stopwords

In [35]:
scores_df = pd.read_csv('FinalData/vadar_scores.csv', index_col=0)
mean_scores = pd.read_csv('FinalData/vadar_mean_compound_scores.csv', index_col=0)

In [36]:
scores_df

,headline,date,neg,neu,pos,compound
0,Nikkei down 1 4 pct on economy fears autos drop,2008-10-02,0.450,0.550,0.000,-0.5994
1,FOREX Dollar poised for biggest weekly gain in...,2008-10-03,0.000,0.597,0.403,0.6597
2,GLOBAL MARKETS Stocks rally on Wells Wachovia ...,2008-10-03,0.000,0.645,0.355,0.5267
3,GLOBAL MARKETS Stocks hold up after Wells Wach...,2008-10-03,0.000,0.800,0.200,0.2500
4,GLOBAL MARKETS U S bailout plan passes inves...,2008-10-03,0.136,0.583,0.282,0.3612
...,...,...,...,...,...,...
221508,Why T Mobile s deal with Sprint could be the w...,2020-02-13,0.000,0.881,0.119,0.2263
221509,6 Low Price to Book Stocks With Great Growth P...,2020-02-13,0.131,0.313,0.556,0.7783
221510,Cisco Systems CSCO Q2 Earnings And Revenues ...,2020-02-13,0.000,1.000,0.000,0.0000
221511,Improve Your Retirement Income With These 3 To...,2020-02-13,0.000,0.701,0.299,0.5719


In [37]:
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [38]:
# Helper Functions 
def getTickerData(ticker, start, end):
  # exception handling for unexpected errors
  try:
    df = web.DataReader(ticker, 'yahoo', start, end)
    # convert the web data to csv format
    df.to_csv('{}.csv'.format(ticker), index=True)
  except:
    print("Error in getting Data...")
def get_data(start_date, end_date, company, stkMarket=""):
  # Ticker Information  
  # Ticker Examples = ['MMM',  'ABT',  'AAPL',  'AMAT',  'APTV',  'ADM',  'ARNC',  'AMZN', 'GOOG']   
  # Stock data retrieval using yahoo
  ticker = company
  stockMarket = stkMarket
  if stockMarket:
    ticker = ticker + "." + stockMarket
    print("Ticker = ",ticker)
  print(f"Collecting the required data for {ticker} from dates: {start_date} to {end_date}...")
  getTickerData(ticker, start_date, end_date)
  df = web.DataReader(ticker, 'yahoo', start_date, end_date)
  df = pd.read_csv('{}.csv'.format(ticker))
  return df

In [39]:
# setting start and end date for stock data retrieval
company = 'SP500'
start_date = date(2008,1,1)
end_date = date(2021,1,1)

sdf = web.DataReader(['sp500'], 'fred', start_date, end_date)
sdf['date'] = sdf.index
sdf = sdf.rename(columns = {'sp500':'Adj Close'})
sdf = sdf.reset_index(drop=True)
sdf
# fetching data using ticker
#sdf = get_data(start_date,end_date,company)
#sdf

,Adj Close,date
0,1319.68,2011-04-15
1,1305.14,2011-04-18
2,1312.62,2011-04-19
3,1330.36,2011-04-20
4,1337.38,2011-04-21
...,...,...
2531,3735.36,2020-12-28
2532,3727.04,2020-12-29
2533,3732.04,2020-12-30
2534,3756.07,2020-12-31


In [40]:
sdf = sdf.dropna()
sdf = sdf.reset_index(drop=True)
sdf

,Adj Close,date
0,1319.68,2011-04-15
1,1305.14,2011-04-18
2,1312.62,2011-04-19
3,1330.36,2011-04-20
4,1337.38,2011-04-21
...,...,...
2440,3703.06,2020-12-24
2441,3735.36,2020-12-28
2442,3727.04,2020-12-29
2443,3732.04,2020-12-30


In [41]:
sdf['date'] = pd.to_datetime(sdf['date'])
sdf = sdf.sort_values(by=['date'], ascending=True)
sdf = sdf.reset_index(drop=True)
sdf

,Adj Close,date
0,1319.68,2011-04-15
1,1305.14,2011-04-18
2,1312.62,2011-04-19
3,1330.36,2011-04-20
4,1337.38,2011-04-21
...,...,...
2440,3703.06,2020-12-24
2441,3735.36,2020-12-28
2442,3727.04,2020-12-29
2443,3732.04,2020-12-30


In [48]:
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [49]:
mean_scores['date'] = pd.to_datetime(mean_scores['date'])
mean_scores = mean_scores.sort_values(by=['date'], ascending=True)
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,mean_compound,date
0,-0.599400,2008-10-02
1,0.449400,2008-10-03
2,-0.655600,2008-10-06
3,-0.460450,2008-10-07
4,0.223500,2008-10-08
...,...,...
3636,0.074910,2020-02-09
3637,0.097086,2020-02-10
3638,0.005268,2020-02-11
3639,0.041805,2020-02-12


In [50]:
sdf.shape, mean_scores.shape

((2445, 2), (3641, 2))

In [45]:
#sdf = sdf.drop(columns=['High', 'Low','Open','Close','Volume'])
#sdf

In [46]:
#sdf['date'] = sdf['Date']
#sdf.drop(columns=['Date'],inplace=True)
#sdf

In [51]:
merged_inner = pd.merge(left=mean_scores, right=sdf, left_on='date', right_on='date')
merged_inner

,mean_compound,date,Adj Close
0,0.009717,2011-04-15,1319.68
1,-0.311260,2011-04-18,1305.14
2,-0.030475,2011-04-19,1312.62
3,0.124682,2011-04-20,1330.36
4,0.045923,2011-04-21,1337.38
...,...,...,...
2215,0.027369,2020-02-07,3327.71
2216,0.097086,2020-02-10,3352.09
2217,0.005268,2020-02-11,3357.75
2218,0.041805,2020-02-12,3379.45


In [52]:
merged_inner = merged_inner.rename(columns = {'Adj Close': 'adj_close'})
df_final = merged_inner[['date','adj_close','mean_compound']]
df_final

,date,adj_close,mean_compound
0,2011-04-15,1319.68,0.009717
1,2011-04-18,1305.14,-0.311260
2,2011-04-19,1312.62,-0.030475
3,2011-04-20,1330.36,0.124682
4,2011-04-21,1337.38,0.045923
...,...,...,...
2215,2020-02-07,3327.71,0.027369
2216,2020-02-10,3352.09,0.097086
2217,2020-02-11,3357.75,0.005268
2218,2020-02-12,3379.45,0.041805


In [54]:
df_name = "FinalData/stock_and_news_"+company+".csv"
df_final.to_csv(df_name)